In [1]:
import requests
import re

In [2]:
def number_of_last_article_in_base():
    # Пока функция читает последний номер из файла
    # В дальнейшем номер будет забираться из redis
    f = open('last_number.txt')
    number = int(f.read())
    return number
    
def number_of_last_article_in_Habr():
    main_url = "https://habr.com/ru/all/"
    r = requests.get(main_url)
    raw_html = str(r.content)
    
    # На главной Хабра найдём номера всех статей
    match = re.findall(r"<article id=\"\d+\"", raw_html)
    
    # Последняя отображаемая статья необязательно самая последняя на сайте
    # Например, если статья была создана давно, но опубликована только что
    # Поэтому с главной страницы берётся максимальный номер
    # Числа +13 и -1 отрезают от pattern часть, не связанную с номером
    
    number = max([int(string[13:-1]) for string in match])
    return number

def send_article(num,data):
    # Пока статьи кладутся просто рядом со скриптом в файл
    # Где название файла - номер статьи
    filename = str(num)+'.txt'
    with open(filename, "w") as file:
        file.write(data)
        
def clean_article_from_tags(raw_article):
    # Первый момент:
    # Еще нужно удалить некоторые специальные тэги, типа /xa0
    # Нужно небольшое исследование провести какие еще тэги могут быть
    # Скорее всего проще всего создать статью, куда включить все возможные
    # элементы разметки. Либо найти статью в блоге Хабра, помню была у них
    # статья, где они презентовали редактор с этой разметкой
    # -------------------------
    # Второй момент:
    # Сейчас я забираю только текст статьи, но думаю, что нужно
    # еще добавить название статьи
    # -------------------------
    # Третий момент:
    # Иногда превью копирует начало основного текста
    # Иногда это совершенно новый текст
    # Иногда это немного модифицированный текст начала статьи
    # Наверное, самый простой вариант взять превью и добавить его перед началом статьи
    # Даже если это будет полное дублирование, то вроде бы ничего страшного
    raw_article_text = raw_article.json()["textHtml"]
    article = re.sub(r'</?[^>]+>|\r\n', '', raw_article_text)
    return article
    
def get_new_articles_from_habr():
    min = number_of_last_article_in_base()
    max = number_of_last_article_in_Habr()
    
    if min >= max:
        pass
    else:
        api_url = "https://habr.com/kek/v2/articles/"
        for num in range(min,max+1):
            post_url = api_url+str(num)
            raw_article = requests.get(post_url)
            if raw_article.status_code == 200:
                article = clean_article_from_tags(raw_article)
                send_article(num,article)
    
if __name__ == "__main__":
    get_new_articles_from_habr()